<a href="https://colab.research.google.com/github/fanbinzj/aitrader/blob/master/my_stock_test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x

In [3]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader
import plotly.express as px

from tqdm import tqdm_notebook, tqdm
from collections import deque

tf.__version__

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


'2.2.0-rc3'

In [0]:
class AI_Trader():
  
  def __init__(self, state_size, action_space=3, model_name="AITrader"): #Stay, Buy, Sell
    
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen=2000)
    self.inventory = []
    self.model_name = model_name
    
    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995
    
    self.model = self.model_builder()
    
  def model_builder(self):
    
    model = tf.keras.models.Sequential()
    
    model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))
    
    model.add(tf.keras.layers.Dense(units=64, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=128, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))
    
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.001))
    
    return model
  
  def trade(self, state):
    
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
    
    actions = self.model.predict(state)
    return np.argmax(actions[0])
  
  
  def batch_train(self, batch_size):
    
    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])
      
    for state, action, reward, next_state, done in batch:
      reward = reward
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
        
      target = self.model.predict(state)
      target[0][action] = reward
      
      self.model.fit(state, target, epochs=1, verbose=0)
      
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay

In [0]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

def dataset_loader(stock_name):
  
  #Complete the dataset loader function
  dataset = data_reader.DataReader(stock_name, data_source="yahoo")
  
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  
  close = dataset['Close']

  return close

def state_creator(data, timestep, window_size):
  
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])
    
  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
  return np.array([state])



In [0]:
stock_name = "AAPL"
data = dataset_loader(stock_name)

In [0]:
window_size = 10

batch_size = 32
data_samples = len(data) - 1


In [8]:
trader = AI_Trader(window_size)
trader.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


In [0]:
plotX = []
plotY = []
counter = 0
def appendPlotValue(totalProfit, arrayIndex):
  plotY.append(totalProfit)
  plotX.append(arrayIndex)

def showProfitPlot():
  fig = px.line(x = plotX, y = plotY, title='profit line')
  fig.show()

In [10]:
data[1]

129.6699981689453

In [0]:
max_lots_limit = 3
episodes = 100

for episode in range(1, episodes + 1):
  
  maxLots = 0
  plotX.clear()
  plotY.clear()
  print("Episode: {}/{}".format(episode, episodes))
  
  state = state_creator(data, 0, window_size + 1)
  
  total_profit = 0
  trader.inventory = []
  
  for t in tqdm(range(data_samples)):
    

    action = trader.trade(state)
    
    next_state = state_creator(data, t+1, window_size + 1)
    reward = 0

    maxLots = max(len(trader.inventory), maxLots)
    
    if action == 1 and len(trader.inventory) < max_lots_limit: #Buying
      trader.inventory.append(data[t])
      print("AI Trader bought at: ", stocks_price_format(data[t]))
      print("TOTAL PROFIT: {}, maxLots = {}".format(total_profit, maxLots))
      
    elif action == 2 and len(trader.inventory) > 0: #Selling
      buy_price = trader.inventory.pop(0)
      
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price

      appendPlotValue(total_profit, t)
      print("AI Trader sold at: ", stocks_price_format(data[t]), ", cur profit: " +stocks_price_format(data[t] - buy_price))
      
    if t == data_samples - 1:
      done = True
    else:
      done = False
      
    trader.memory.append((state, action, reward, next_state, done))
    
    state = next_state
    
    if done:
      print("########################")
      print("TOTAL PROFIT: {}, maxLots = {}".format(total_profit, maxLots))
      print("########################")
      #print(plotY)
      showProfitPlot()
    
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
      
  if episode % 5 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))

  0%|          | 0/1257 [00:00<?, ?it/s]

Episode: 1/100
AI Trader bought at:  $ 128.619995
TOTAL PROFIT: 0, maxLots = 0
AI Trader bought at:  $ 129.669998
TOTAL PROFIT: 0, maxLots = 1
AI Trader sold at:  $ 130.559998 , cur profit: $ 1.940002
AI Trader bought at:  $ 128.639999
TOTAL PROFIT: 1.94000244140625, maxLots = 2
AI Trader bought at:  $ 125.150002
TOTAL PROFIT: 1.94000244140625, maxLots = 2
AI Trader sold at:  $ 128.949997 , cur profit: - $ 0.720001
AI Trader sold at:  $ 128.699997 , cur profit: $ 0.059998
AI Trader sold at:  $ 125.800003 , cur profit: $ 0.650002
AI Trader bought at:  $ 128.949997
TOTAL PROFIT: 1.9300003051757812, maxLots = 3
AI Trader bought at:  $ 128.770004
TOTAL PROFIT: 1.9300003051757812, maxLots = 3
AI Trader bought at:  $ 130.070007
TOTAL PROFIT: 1.9300003051757812, maxLots = 3
AI Trader sold at:  $ 130.059998 , cur profit: $ 1.110001
AI Trader bought at:  $ 131.389999
TOTAL PROFIT: 3.0400009155273438, maxLots = 3
AI Trader sold at:  $ 132.539993 , cur profit: $ 3.769989
AI Trader sold at:  $ 132

  3%|▎         | 34/1257 [00:06<14:20,  1.42it/s]

AI Trader bought at:  $ 128.880005
TOTAL PROFIT: 2.0599746704101562, maxLots = 3


  3%|▎         | 35/1257 [00:08<22:26,  1.10s/it]

...

 29%|██▉       | 363/1257 [13:01<31:58,  2.15s/it]

AI Trader sold at:  $ 113.949997 , cur profit: $ 1.239998


 29%|██▉       | 364/1257 [13:03<31:44,  2.13s/it]

AI Trader sold at:  $ 112.180000 , cur profit: - $ 0.699997


 30%|██▉       | 371/1257 [13:17<31:21,  2.12s/it]